In [195]:
# !pip install shapely

In [196]:
import json
import webbrowser
import pandas as pd
import folium
import requests
from shapely.geometry import shape, Point # 지리 공간 데이터 사용

In [197]:
df = pd.read_csv("C:/Sosang/6.개폐업률 지도시각화/외식업_개폐업수률(2022).csv", encoding='euc-kr', thousands=',')

In [198]:
df

,행정구역/외식업,개업수,폐업수,개업률,폐업률
0,종로구,833,760,2.975,2.750
1,중구,686,696,2.625,2.650
2,용산구,767,585,3.550,2.700
3,성동구,729,630,3.975,3.425
4,광진구,851,796,3.975,3.675
5,동대문구,716,684,3.575,3.375
6,중랑구,677,673,3.875,3.850
7,성북구,679,617,3.500,3.200
8,강북구,592,608,3.750,3.850
9,도봉구,430,438,3.475,3.550


In [199]:
seoul_group_data=df[0:][['행정구역/외식업','개업률']]

In [200]:
seoul_group_data

,행정구역/외식업,개업률
0,종로구,2.975
1,중구,2.625
2,용산구,3.550
3,성동구,3.975
4,광진구,3.975
5,동대문구,3.575
6,중랑구,3.875
7,성북구,3.500
8,강북구,3.750
9,도봉구,3.475


In [201]:
seoul_open=df.sort_values(by='개업률', ascending=False)[:10]
seoul_close=df.sort_values(by='폐업률', ascending=False)[:10]

In [202]:
print(seoul_open)
print(seoul_close)

   행정구역/외식업   개업수   폐업수    개업률    폐업률
15      강서구  1313  1226  4.550  4.250
13      마포구  1680  1439  4.300  3.700
23      송파구  1428  1357  4.025  3.850
3       성동구   729   630  3.975  3.425
4       광진구   851   796  3.975  3.675
17      금천구   582   547  3.900  3.650
6       중랑구   677   673  3.875  3.850
24      강동구   823   882  3.800  4.100
20      관악구   899   992  3.775  4.175
8       강북구   592   608  3.750  3.850
   행정구역/외식업   개업수   폐업수    개업률    폐업률
15      강서구  1313  1226  4.550  4.250
20      관악구   899   992  3.775  4.175
24      강동구   823   882  3.800  4.100
14      양천구   623   668  3.750  4.025
23      송파구  1428  1357  4.025  3.850
6       중랑구   677   673  3.875  3.850
8       강북구   592   608  3.750  3.850
11      은평구   725   736  3.700  3.750
13      마포구  1680  1439  4.300  3.700
4       광진구   851   796  3.975  3.675


In [203]:
seoul_map = folium.Map(location=[37.5502, 126.982], zoom_start=12)

# 서울시 행정구역 위도 경도값 GeoJSON 데이터 불러오기
jpath = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
seoul_geo = json.loads(requests.get(jpath).text) # JSON 문자열을 파이썬 데이터로 변환하여 저장


In [204]:
len(seoul_geo['features'][0]['geometry']['coordinates'][0])

44

In [205]:
seoul_geo['features'][0]['geometry']['coordinates'][0][15]

#_______feature_________

[127.18169407550688, 37.56286338914073]

In [206]:
# GeoJson 객체 생성 및 서울 지도에 추가
folium.GeoJson(
    seoul_geo,
    name='geojson'
).add_to(seoul_map)

# Choropleth 맵 생성 및 추가
folium.Choropleth(
    geo_data=seoul_geo,
    data=seoul_group_data,
    columns=['행정구역/외식업', '개업률'],
    key_on='feature.properties.name',  # GeoJSON 데이터의 키값 지정 (name이 지역명)
    fill_color='PuBu',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='구별 개업률',
    highlight=True
).add_to(seoul_map)


# 데이터프레임의 값을 GeoJSON 데이터에 매핑하여 마커로 표시
for feature in seoul_geo['features']: # 'features'라는 키값 아래의 정보를 추출
    properties = feature['properties']
    city_name = properties['name']
    
    # 데이터프레임에서 해당 지역의 점포수 값 가져오기
    value = seoul_group_data[seoul_group_data['행정구역/외식업'] == city_name]['개업률'].values 
    if len(value) > 0:
        value = value[0]
    else:
        value = 'N/A'
    
    '''
    # GeoJSON 데이터에서 위도와 경도 추출
    coordinates = feature['geometry']['coordinates']
    latitude = coordinates[0][15][1] if coordinates else None
    longitude = coordinates[0][15][0] if coordinates else None
    lat= coordinates[0][20][1]
    lon = coordinates[0][20][0]
    '''
    # lat= feature['geometry']['coordinates'][0][15][1]
    # lon = feature['geometry']['coordinates'][0][15][0]
    # 행정구역의 중간 지점 계산
    geom = shape(feature['geometry'])
    center_point = geom.centroid
    
   
    # 중간 지점에 텍스트로 지역명과 점포수 표시
    folium.Marker(
        location=[center_point.y, center_point.x],
        icon=folium.DivIcon(html=f'<div style="font-size: 12px; color: black; white-space: nowrap;">{city_name}<br/>{value}</div>')
    ).add_to(seoul_map)
    
    # icon_fa_list=['battery-empty','battery-full','battery-half','battery-quarter','battery-three-quarters']
    
    if city_name in ['성동구', '금천구']:
        icon_fa = folium.Icon(color='black',prefix='fa')
        tooltip_a='개업률이 높은데 폐업률 낮은 곳'
    elif city_name in ['마포구', '송파구']:
        icon_fa = folium.Icon(color='blue', prefix='fa')
        tooltip_a='트렌드(5만 이상)'
    elif city_name in ['광진구', '관악구']:
        icon_fa = folium.Icon(color='orange',prefix='fa')
        tooltip_a='대학가 있는 곳'
    elif city_name in ['중랑구','강북구']:
        icon_fa = folium.Icon(color='green',prefix='fa')
        tooltip_a='생활인구수 적은 곳'
    elif city_name in ['강동구']:
        icon_fa = folium.Icon(color='pink', prefix='fa')
        tooltip_a='신도시 유무(무)'
    elif city_name in ['강서구']:
        icon_fa = folium.Icon(color='red',prefix='fa')
        tooltip_a='최종확정'
    else: 
        #icon_fa = folium.Icon(color='blue',icon=icon_fa_list[1], prefix='fa')
        continue

    folium.Marker(
    location=[center_point.y+0.001, center_point.x+0.001],
    # target=_'blink' 옵션으로 새 창을 띄움. 없으면 현재 html 결과창에서 창이 열림
    icon = icon_fa,
    popup="<a href='링크' target=_'blik'>사이트</a>", 
    # 커서가 마커를 가리키면 설명글 같이 띄움.
    tooltip= tooltip_a
    ).add_to(seoul_map)
    
    
seoul_map
# 지도 저장 및 열기
seoul_map.save('C:/Sosang/6.개폐업률 지도시각화/test.html')
webbrowser.open_new("C:/Sosang/6.개폐업률 지도시각화/test.html")



True

In [207]:
icon_fa_list=['battery-empty','battery-full','battery-half','battery-quarter','battery-three-quarters']

In [208]:
icon_fa_list[0]

'battery-empty'